In [21]:
# En esta celda se puede probar la funcion de analisis de una imagen

from lib.image import image_data
from lib.analysis import dyssynchonia_analysis

def analisis_calcio(image_path, min_dist_between_maxs, ancho_corte=5):
    image = image_data.get_image_data(image_path)

    # Select ROI
    fromCenter = False
    showCrosshair = False
    seleted_parameters = cv2.selectROI(image, fromCenter, showCrosshair)

    # Crop image
    x_start = int(seleted_parameters[1])
    x_end = x_start + int(seleted_parameters[3])
    y_start = int(seleted_parameters[0])
    y_end = y_start + int(seleted_parameters[2])

    image = image_data.crop_vertical(image, x_start, x_end)
    image = image_data.crop_horizontal(image, y_start, y_end)

    return dyssynchonia_analysis.analyze_image(image, min_dist_between_maxs, ancho_corte)


# Path completo donde esta la imagen (incluyendo nombre y extencion)
path = '/home/agusgs/Projects/bioinfo/Calcium_analizer/c3e000.tif'

# Ancho de la feta a analisar (es un parametro opcional)
ancho_corte = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 20

analisis_calcio(path, min_dist_between_maxs, ancho_corte)


{'hole_image': {'intensities': array([2149, 2143, 2132, 2128, 2097, 2089, 2092, 2070, 2046, 2025, 2005,
         1995, 1975, 1952, 1930, 1910, 1881, 1848, 1828, 1809, 1791, 1752,
         1719, 1700, 1682, 1653, 1625, 1598, 1562, 1527, 1496, 1469, 1429,
         1402, 1364, 1346, 1312, 1260, 1237, 1211, 1174, 1137, 1112, 1096,
         1060, 1022,  993,  984,  967,  917,  896,  883,  865,  840,  810,
          791,  786,  787,  791,  791,  794,  792,  801,  836,  867,  901,
          947,  980, 1041, 1090, 1169, 1221, 1295, 1362, 1454, 1514, 1595,
         1651, 1728, 1788, 1854, 1916, 1964, 2013, 2057, 2090, 2110, 2141,
         2159, 2165, 2171, 2175, 2164, 2160, 2145, 2132, 2129, 2119, 2099,
         2082, 2058, 2036, 2012, 1997, 1987, 1973, 1947, 1922, 1907, 1891,
         1874, 1851, 1831, 1804, 1781, 1756, 1746, 1707, 1669, 1639, 1624,
         1602, 1577, 1538, 1501, 1482, 1441, 1399, 1362, 1335, 1290, 1251,
         1225, 1196, 1149, 1109, 1095, 1061, 1017,  984,  969,  937,  8